In [12]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import filter_ops

import matplotlib.pyplot as plt
%matplotlib inline

import pickle
import numpy as np
import timeit

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

import datetime
def ts(fmt='%d-%m-%y-%H:%M:%S'):
    return datetime.datetime.now().strftime(fmt)

def tf_config(devices='0'):
      return tf.ConfigProto(allow_soft_placement=False,
                            gpu_options=tf.GPUOptions(allow_growth=True,
                                                      visible_device_list=devices))

In [13]:
def relative_error(x, y, norm=np.abs, eps=1e-6):
    return norm(x - y) / (np.maximum(norm(x), norm(y)) + eps)

In [4]:
outputs = {}

In [43]:
# tf.reset_default_graph()

# np.random.seed(0)
# N, C, F = 64, 8, 3
# probs_shape = [N*N, C]
# np_probs = np.random.normal(size=probs_shape).astype(np.float32)
# features_shape = [N*N, F]
# # TODO: 
# np_features = np.random.normal(0, 1, size=features_shape).astype(np.float32)

# probs = tf.convert_to_tensor(np_probs)
# features = tf.convert_to_tensor(np_features)
# lattice = filter_ops.permuto_init(features)
# #impl_type = 'gpu'    
# impl_type = 'cpu'    
# with tf.device(f'/{impl_type}:0'):
#     out = filter_ops.permuto_compute(np_probs, *lattice)
    
# with tf.Session() as sess:
#     print(f'[{ts()}] running forward pass')
#     outputs[impl_type] = sess.run(out)
#     print(f'[{ts()}] done!')
    
# #     err = tf.test.compute_gradient_error(probs, probs_shape, out, probs_shape)
# #     g_thr, g_num = tf.test.compute_gradient(probs, probs_shape, out, probs_shape)
# #     print(f'max gradient error: {err}')
# #     rel_err = relative_error(g_thr, g_num, norm=np.linalg.norm).max()
# #     print(f'max gradient relative error: {rel_err}')   


[31-01-18-15:44:03] running forward pass
[31-01-18-15:44:03] done!


In [14]:
tf.reset_default_graph()
np.random.seed(0)
N, C, F = 64, 8, 3
probs_shape = [N*N, C]
np_probs = np.random.normal(size=probs_shape).astype(np.float32)
features_shape = [N*N, F]
np_features = np.random.normal(0, 1, size=features_shape).astype(np.float32)

probs = tf.convert_to_tensor(np_probs)
features = tf.convert_to_tensor(np_features)
lattice = filter_ops.permuto_init(features)

In [15]:
lattice

PermutoInit(offsets=<tf.Tensor 'PermutoInit:0' shape=(48, 4) dtype=int32>, weights=<tf.Tensor 'PermutoInit:1' shape=(48, 4) dtype=float32>, neighbours=<tf.Tensor 'PermutoInit:2' shape=(?, 4, 2) dtype=int32>)

In [17]:
with tf.Session(config=tf_config()) as sess:
    _lattice = sess.run(lattice)

In [20]:
_lattice.neighbours.shape

(339, 4, 2)

In [ ]:
def meanfield_iteration(q, unaries, lattice):
    """a single iteration of MF inference"""
    # pairwise "messages"
    pairwise = filter_ops.permuto_compute(q, *lattice)

    pass


## test

In [6]:
import numpy as np
import pydensecrf.densecrf as dcrf

N, C, F = 64, 8, 3
np.random.seed(0)
probs_shape = [N*N, C]
np_probs = np.random.normal(size=probs_shape).astype(np.float32)
features_shape = [N*N, F]
np_features = np.random.normal(0, 1, size=features_shape).astype(np.float32)

crf = dcrf.DenseCRF(N*N, C)
crf.setUnaryEnergy(np_probs.T.copy())
crf.addPairwiseEnergy(np_features.T.copy(), 
                      1.0,
                      dcrf.CONST_KERNEL,
                      dcrf.NO_NORMALIZATION)
q, tmp1, tmp2 = crf.startInference()
crf.stepInference(q, tmp1, tmp2)
print(np.array(q).T)

[[  5.86433150e-02   1.89400449e-01   1.19181640e-01 ...,   2.55714864e-01
    7.15733543e-02   2.69222111e-01]
 [  1.17280884e-02   5.35196438e-03   2.50127047e-01 ...,   1.52391821e-01
    2.48269543e-01   2.51372427e-01]
 [  1.83028751e-04   7.37607619e-03   4.21964750e-02 ...,   1.99468061e-02
    1.76119953e-02   8.42108548e-01]
 ..., 
 [  2.40331404e-02   5.13923215e-03   2.04230666e-01 ...,   9.36731324e-03
    2.15947688e-01   4.74493802e-01]
 [  3.66344452e-02   4.01698835e-02   4.29359466e-01 ...,   1.40301846e-02
    1.22822402e-02   4.25423771e-01]
 [  6.11266660e-05   2.09598657e-04   3.34815383e-02 ...,   3.17112952e-02
    1.64064090e-03   9.24607098e-01]]


In [8]:
np.array(q).T.shape

(4096, 8)

In [49]:
for it in ['cpu', 'gpu']:
    print(f'max error = {np.max(np.array(q).T - outputs[it])}')

max error = 0.0
max error = 0.0


In [26]:
def np_softmax(logits, axis=None):
    l_max = np.max(logits, axis=axis, keepdims=True)
    l_exp = np.exp(logits - l_max)
    return l_exp / np.sum(l_exp, axis=axis, keepdims=True)

def np_pdist(X):
    """computes pairwise distances
    Args:
        X - [N, D] matrix of N points
    Returns:
        [N, N] matrix
    """
    X2 = np.sum(np.square(X), axis=1, keepdims=True)
    return X2 - 2 * np.dot(X, X.T) + X2.T

#q_in = np_softmax(-np.ones([N*N, C]), axis=1)
dists = np_pdist(np_features)
kernels = np.exp(-dists / 2.0) 
msgs = np.matmul(kernels, np_probs)
print(msgs)

[[ -1.32233858  -1.22623873   1.18854332 ...,  -3.55239964   1.46188354
   -5.01913691]
 [ -8.3676424   -9.25286961  -6.0343504  ...,  -7.51579475  -8.66574097
  -13.19216824]
 [  1.53489137  -1.75268912   4.22882366 ...,   1.98597836   4.20744038
   -6.21151829]
 ..., 
 [ -1.02936697  -1.78678703   1.35178769 ..., -11.20064735   2.3003726
   -3.34708738]
 [ -0.99986696   0.24824369   1.07701159 ...,   2.9248414   -0.18105268
   -2.16538239]
 [  5.52563763   4.48496437   4.42841959 ...,   3.40074778  -6.25938272
  -13.26993847]]


In [35]:
np.abs(outputs['cpu'] - msgs)

12.487908

In [ ]:


def np_q_update(q_in, unaries, features, compat):
    dists = np_pdist(np_features)
    kernels = np.exp(-dists / 2.0)
    msgs = np.matmul(kernels, q_in) - q_in
    q_up = np_softmax(unaries - compat * msgs, axis=1) 
    return q_up

q_in = np_softmax(-np_probs, axis=1)
q_up = np_q_update(q_in, -np_probs, np_features, 1.0)

# print(np.mean(np.abs(q_up - np.array(q).T)))
# print(np.mean(np.abs(q_in - np.array(q).T)))

## reference implementation

In [ ]:
from scipy.stats import multivariate_normal

H, W, NLABELS = 400, 512, 2

# This creates a gaussian blob...
pos = np.stack(np.mgrid[0:H, 0:W], axis=2)
rv = multivariate_normal([H//2, W//2], (H//4)*(W//4))
probs = rv.pdf(pos)

# ...which we project into the range [0.4, 0.6]
probs = (probs-probs.min()) / (probs.max()-probs.min())
probs = 0.5 + 0.2 * (probs-0.5)

# The first dimension needs to be equal to the number of classes.
# Let's have one "foreground" and one "background" class.
# So replicate the gaussian blob but invert it to create the probability
# of the "background" class to be the opposite of "foreground".
probs = np.tile(probs[np.newaxis,:,:],(2,1,1))
probs[1,:,:] = 1 - probs[0,:,:]

# Let's have a look:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.imshow(probs[0,:,:])
plt.title('Foreground probability')
plt.axis('off')
plt.colorbar()
plt.subplot(1,2,2) 
plt.imshow(probs[1,:,:])
plt.title('Background probability')
plt.axis('off')
plt.colorbar()


# vanilla densecrf
import pydensecrf.densecrf as dcrf
from pydensecrf.utils import unary_from_softmax, create_pairwise_bilateral

# Inference without pair-wise terms
U = unary_from_softmax(probs)  # note: num classes is first dim
d = dcrf.DenseCRF2D(W, H, NLABELS)
d.setUnaryEnergy(U)

# Run inference for 10 iterations
Q_unary = d.inference(10)

# The Q is now the approximate posterior, we can get a MAP estimate using argmax.
map_soln_unary = np.argmax(Q_unary, axis=0)

# Unfortunately, the DenseCRF flattens everything, so get it back into picture form.
map_soln_unary = map_soln_unary.reshape((H,W))

# And let's have a look.
plt.imshow(map_soln_unary); plt.axis('off'); plt.title('MAP Solution without pairwise terms');


NCHAN=1

# Create simple image which will serve as bilateral.
# Note that we put the channel dimension last here,
# but we could also have it be the first dimension and
# just change the `chdim` parameter to `0` further down.
img = np.zeros((H,W,NCHAN), np.uint8)
img[H//3:2*H//3,W//4:3*W//4,:] = 1

plt.imshow(img[:,:,0]); plt.title('Bilateral image'); plt.axis('off'); plt.colorbar();

# Create the pairwise bilateral term from the above image.
# The two `s{dims,chan}` parameters are model hyper-parameters defining
# the strength of the location and image content bilaterals, respectively.
pairwise_energy = create_pairwise_bilateral(sdims=(10,10), schan=(0.01,), img=img, chdim=2)

In [ ]:
# pairwise_energy now contains as many dimensions as the DenseCRF has features,
# which in this case is 3: (x,y,channel1)
img_en = pairwise_energy.reshape((-1, H, W))  # Reshape just for plotting
plt.figure(figsize=(15,5))
plt.subplot(1,3,1); plt.imshow(img_en[0]); plt.title('Pairwise bilateral [x]'); plt.axis('off'); plt.colorbar();
plt.subplot(1,3,2); plt.imshow(img_en[1]); plt.title('Pairwise bilateral [y]'); plt.axis('off'); plt.colorbar();
plt.subplot(1,3,3); plt.imshow(img_en[2]); plt.title('Pairwise bilateral [c]'); plt.axis('off'); plt.colorbar();


d = dcrf.DenseCRF2D(W, H, NLABELS)
d.setUnaryEnergy(U)
d.addPairwiseEnergy(pairwise_energy, compat=10)  # `compat` is the "strength" of this potential.

# This time, let's do inference in steps ourselves
# so that we can look at intermediate solutions
# as well as monitor KL-divergence, which indicates
# how well we have converged.
# PyDenseCRF also requires us to keep track of two
# temporary buffers it needs for computations.
Q, tmp1, tmp2 = d.startInference()
for _ in range(5):
    d.stepInference(Q, tmp1, tmp2)
kl1 = d.klDivergence(Q) / (H*W)
map_soln1 = np.argmax(Q, axis=0).reshape((H,W))

for _ in range(20):
    d.stepInference(Q, tmp1, tmp2)
kl2 = d.klDivergence(Q) / (H*W)
map_soln2 = np.argmax(Q, axis=0).reshape((H,W))

for _ in range(50):
    d.stepInference(Q, tmp1, tmp2)
kl3 = d.klDivergence(Q) / (H*W)
map_soln3 = np.argmax(Q, axis=0).reshape((H,W))

img_en = pairwise_energy.reshape((-1, H, W))  # Reshape just for plotting
plt.figure(figsize=(15,5))
plt.subplot(1,3,1); plt.imshow(map_soln1);
plt.title('MAP Solution with DenseCRF\n(5 steps, KL={:.2f})'.format(kl1)); plt.axis('off');
plt.subplot(1,3,2); plt.imshow(map_soln2);
plt.title('MAP Solution with DenseCRF\n(20 steps, KL={:.2f})'.format(kl2)); plt.axis('off');
plt.subplot(1,3,3); plt.imshow(map_soln3);
plt.title('MAP Solution with DenseCRF\n(75 steps, KL={:.2f})'.format(kl3)); plt.axis('off');